In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dense
import keras_nlp

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import copy
warnings.simplefilter(action='ignore')

In [4]:
# Load the MovieLens 100K dataset
# Assuming the file is 'u.data' in the same directory
# Columns: user_id, movie_id, rating, timestamp
data = pd.read_csv('u.data', sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Step 0: only select top 50 movies
valid_movies = np.array(data['movie_id'].value_counts().index[:50])
data = data[data['movie_id'].isin(valid_movies)]

# Step 1: Remove users s.t. count >= 2 * unique timestamp
temp = data.groupby('user_id')['timestamp'].agg(['count', 'nunique']).reset_index()
valid_users = temp[temp['count']/temp['nunique'] < 2]['user_id']
data = data[data['user_id'].isin(valid_users)]

# Step 2: Deduplicate and recode
data = data.groupby(['user_id', 'timestamp'], group_keys=False).apply(lambda group: group.sample(n=1, random_state=42))
data = data.sort_values(['user_id', 'timestamp']).reset_index(drop = True)
data['movie_id'] = pd.factorize(data['movie_id'])[0] + 1

num_users = data['user_id'].nunique()
num_movies = data['movie_id'].nunique()
num_ratings = len(data)

sparsity = 1 - (num_ratings / (num_users * num_movies))

print(f"Filtered Dataset: {num_users} users, {num_movies} movies")
print(f"Sparsity: {sparsity:.2%}")

Filtered Dataset: 902 users, 50 movies
Sparsity: 69.50%


In [5]:
data

,user_id,movie_id,rating,timestamp
0,1,1,5,874965478
1,1,2,5,874965706
2,1,3,5,874965739
3,1,4,5,874965758
4,1,5,5,874965954
...,...,...,...,...
13752,943,23,4,888639407
13753,943,13,5,888639427
13754,943,8,4,888692413
13755,943,41,4,888692699


In [6]:
np.random.seed(42)

all_users = data['user_id'].unique()
shuffled_indices = np.random.permutation(len(all_users))

midpoint = (3 * len(all_users)) // 4
indices_1 = shuffled_indices[:midpoint]
indices_2 = shuffled_indices[midpoint:]

train_users = shuffled_indices[indices_1]
test_users = shuffled_indices[indices_2]

In [7]:
data_train = data[data['user_id'].isin(train_users)].reset_index(drop = True)
data_test = data[data['user_id'].isin(test_users)].reset_index(drop = True)

In [8]:
# Prepare baskets
def prepare_baskets(data):
    return data.groupby("user_id")["movie_id"].apply(list).tolist()

train_baskets = prepare_baskets(data_train)
test_baskets = prepare_baskets(data_test)

train_baskets, val_baskets = train_test_split(train_baskets, test_size=0.25, random_state=42)

max_len = max(len(basket) for basket in train_baskets + val_baskets + test_baskets)

In [9]:
D = 32
batch_size = 256
max_epochs = 2000
lr = 1e-4
max_items = len(set(data_test['movie_id']))

In [10]:
len(set(data_test['movie_id']))

50

In [11]:
def preprocess_baskets(baskets):
    context_inputs = []
    target_inputs = []
    masked_idxs = []

    for basket in baskets:
        for idx, elt in enumerate(basket):
            target_inputs.append(elt)
            context_inputs.append(basket[:idx] + [max_items + 1] + basket[(idx+1):])
            masked_idxs.append(idx)

    context_inputs = pad_sequences(context_inputs, padding='post', maxlen = max_len, value=0)
    return np.array(context_inputs), np.array(target_inputs) - 1, np.array(masked_idxs)

train_context_input, train_target_input, train_masked_idxs = preprocess_baskets(train_baskets)
val_context_input, val_target_input, val_masked_idxs = preprocess_baskets(val_baskets)
test_context_input, test_target_input, test_masked_idxs = preprocess_baskets(test_baskets)

In [12]:
input_context = layers.Input(shape=(max_len,), dtype=tf.int32, name="context_input")
masked_idx_input = layers.Input(shape=(1,), dtype=tf.int32, name="masked_idx_input")

alpha_embedding = layers.Embedding(input_dim=max_items + 2, output_dim=D, name="alpha_embedding")
context_embedding = alpha_embedding(input_context) 

class ZeroMaskEmbedding(layers.Layer):
    def call(self, embeddings, input_tokens):
        mask = tf.cast(tf.not_equal(input_tokens, 0), tf.float32) 
        mask = tf.expand_dims(mask, axis=-1) 
        return embeddings * mask 

context_embedding = ZeroMaskEmbedding()(context_embedding, input_context)

class MaskLayer(layers.Layer):
    def call(self, input_context, position):
        return position * tf.expand_dims(tf.cast(tf.not_equal(input_context, 0), tf.float32), axis = -1)

position = keras_nlp.layers.PositionEmbedding(sequence_length=max_len)(context_embedding)
masked_position = MaskLayer()(input_context, position)
context_embedding = context_embedding + masked_position

attention_layer_1 = MultiHeadAttention(num_heads=2, key_dim=16, name="multi_head_attention_1")
attn_output_1 = attention_layer_1(
    query=context_embedding,
    value=context_embedding,
    key=context_embedding,
    use_causal_mask=True
)

attn_output_1 = context_embedding + attn_output_1

attention_layer_2 = MultiHeadAttention(num_heads=2, key_dim=16, name="multi_head_attention_2")
attn_output_2 = attention_layer_2(
    query=attn_output_1,
    value=attn_output_1,
    key=attn_output_1,
    use_causal_mask=True
)

context_embedding = attn_output_1 + attn_output_2

class GatherLayer(layers.Layer):
    def call(self, inputs):
        context_embedding, masked_idx_input = inputs
        return tf.gather(context_embedding, indices=tf.squeeze(masked_idx_input, axis=-1), batch_dims=1)

masked_embeddings = GatherLayer()([context_embedding, masked_idx_input])

output = layers.Dense(max_items, activation="softmax", name="output_layer", use_bias = False)(masked_embeddings)

model = Model(inputs=[input_context, masked_idx_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), loss="sparse_categorical_crossentropy")

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    [train_context_input, train_masked_idxs], train_target_input,
    validation_data=([val_context_input, val_masked_idxs], val_target_input),
    batch_size=batch_size,
    epochs=max_epochs,
    callbacks=[early_stopping]
)

Epoch 1/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 3.9132 - val_loss: 3.9117
Epoch 2/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.9094 - val_loss: 3.9078
Epoch 3/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.9019 - val_loss: 3.9038
Epoch 4/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.9010 - val_loss: 3.8996
Epoch 5/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.8971 - val_loss: 3.8948
Epoch 6/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.8907 - val_loss: 3.8893
Epoch 7/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.8840 - val_loss: 3.8827
Epoch 8/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.8727 - val_loss: 3.8743
Epoch 9/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.8672 - val_loss: 3.8640
Epoch 10/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.8523 - val_loss: 3.8504
Epoch 11/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.8385 - val_loss: 3.8333
Epoch 12/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 

29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5623 - val_loss: 3.6052
Epoch 69/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5705 - val_loss: 3.6040
Epoch 70/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5626 - val_loss: 3.6033
Epoch 71/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5655 - val_loss: 3.6027
Epoch 72/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5609 - val_loss: 3.6013
Epoch 73/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5582 - val_loss: 3.6006
Epoch 74/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5665 - val_loss: 3.5996
Epoch 75/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5475 - val_loss: 3.5986
Epoch 76/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5585 - val_loss: 3.5977
Epoch 77/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5359 - val_loss: 3.5968
Epoch 78/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.5439 - val_loss: 3.5959
Epoch 79/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20

Epoch 135/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4617 - val_loss: 3.5333
Epoch 136/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4594 - val_loss: 3.5327
Epoch 137/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4364 - val_loss: 3.5320
Epoch 138/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4612 - val_loss: 3.5314
Epoch 139/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4619 - val_loss: 3.5307
Epoch 140/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4498 - val_loss: 3.5296
Epoch 141/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4655 - val_loss: 3.5290
Epoch 142/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4629 - val_loss: 3.5289
Epoch 143/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4455 - val_loss: 3.5278
Epoch 144/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4445 - val_loss: 3.5271
Epoch 145/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4580 - val_loss: 3.5265
Epoch 146/2000
29/29 

29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3961 - val_loss: 3.4932
Epoch 202/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.3994 - val_loss: 3.4924
Epoch 203/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4095 - val_loss: 3.4924
Epoch 204/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4004 - val_loss: 3.4913
Epoch 205/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4160 - val_loss: 3.4915
Epoch 206/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.4018 - val_loss: 3.4908
Epoch 207/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3838 - val_loss: 3.4903
Epoch 208/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 3.3914 - val_loss: 3.4894
Epoch 209/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3977 - val_loss: 3.4893
Epoch 210/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3936 - val_loss: 3.4888
Epoch 211/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3839 - val_loss: 3.4879
Epoch 212/2000
29/29 ━━━━━━━━━━━━━━━

29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3583 - val_loss: 3.4630
Epoch 268/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3428 - val_loss: 3.4623
Epoch 269/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3608 - val_loss: 3.4621
Epoch 270/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3508 - val_loss: 3.4617
Epoch 271/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3381 - val_loss: 3.4615
Epoch 272/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3548 - val_loss: 3.4614
Epoch 273/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3587 - val_loss: 3.4612
Epoch 274/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3586 - val_loss: 3.4608
Epoch 275/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3578 - val_loss: 3.4602
Epoch 276/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3455 - val_loss: 3.4602
Epoch 277/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3464 - val_loss: 3.4600
Epoch 278/2000
29/29 ━━━━━━━━━━━━━━━

29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3165 - val_loss: 3.4506
Epoch 334/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3182 - val_loss: 3.4515
Epoch 335/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3262 - val_loss: 3.4509
Epoch 336/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3173 - val_loss: 3.4503
Epoch 337/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3065 - val_loss: 3.4509
Epoch 338/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3173 - val_loss: 3.4504
Epoch 339/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3059 - val_loss: 3.4509
Epoch 340/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3006 - val_loss: 3.4511
Epoch 341/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3122 - val_loss: 3.4502
Epoch 342/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3065 - val_loss: 3.4502
Epoch 343/2000
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3107 - val_loss: 3.4507
Epoch 344/2000
29/29 ━━━━━━━━━━━━━━━

In [13]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ context_input       │ (None, 44)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ alpha_embedding     │ (None, 44, 32)    │      1,664 │ context_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_mask_embedding │ (None, 44, 32)    │          0 │ alpha_embedding[… │
│ (ZeroMaskEmbedding) │                   │            │ context_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ position_embedding  │ (None, 44, 32)    │      1,408 │ zero_mask_embedd… │
│ (PositionEmbedding) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mask_layer          │ (None, 44, 32)    │          0 │ context_input[0]… │
│ (MaskLayer)         │                   │            │ position_embeddi… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 44, 32)    │          0 │ zero_mask_embedd… │
│                     │                   │            │ mask_layer[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 44, 32)    │      4,224 │ add[0][0],        │
│ (MultiHeadAttentio… │                   │            │ add[0][0],        │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 44, 32)    │          0 │ add[0][0],        │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 44, 32)    │      4,224 │ add_1[0][0],      │
│ (MultiHeadAttentio… │                   │            │ add_1[0][0],      │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 44, 32)    │          0 │ add_1[0][0],      │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ masked_idx_input    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ gather_layer        │ (None, 32)        │          0 │ add_2[0][0],      │
│ (GatherLayer)       │                   │            │ masked_idx_input… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 50)        │      1,600 │ gather_layer[0][… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 39,362 (153.76 KB)

 Trainable params: 13,120 (51.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 26,242 (102.51 KB)

In [15]:
# Evaluate on Test Data
test_loss = model.evaluate([test_context_input, test_masked_idxs], test_target_input, batch_size=batch_size)
print(f"Test Loss: {test_loss}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.4431
Test Loss: 3.443751096725464
